In [ ]:
using ITensors
using MSSTA

In [ ]:
"""
Reconstruct 3D matrix
"""
function _tomat3(a)
    sites = siteinds(a)
    N = length(sites)
    Nreduced = N ÷ 3
    sites_ = [sites[1:3:N]..., sites[2:3:N]..., sites[3:3:N]...]
    return reshape(Array(reduce(*, a), sites_), 2^Nreduced, 2^Nreduced, 2^Nreduced)
end

In [ ]:
nbit = 2
D = 2
sx = [Index(2, "Qubit,x=$n") for n in 1:nbit]
sy = [Index(2, "Qubit,y=$n") for n in 1:nbit]
sz = [Index(2, "Qubit,z=$n") for n in 1:nbit]
sk = [Index(2, "Qubit,k=$n") for n in 1:nbit]

In [ ]:
"""
Matrix multiplication for x, y, z and elementwise multiplication for k

C(x, z, k) = sum_y A(x, y, k) * B(y, z, k)

automul performs elementwise and matrix multiplication for two MPSs
"""
sites_a = collect(Iterators.flatten(zip(sx, sy, sk)))
sites_b = collect(Iterators.flatten(zip(sy, sz, sk)))

a = randomMPS(sites_a; linkdims=D)
b = randomMPS(sites_b; linkdims=D)

# Reference data
a_arr = _tomat3(a)
b_arr = _tomat3(b)
ab_arr = zeros(Float64, 2^nbit, 2^nbit, 2^nbit)
for k in 1:(2^nbit)
    ab_arr[:, :, k] .= a_arr[:, :, k] * b_arr[:, :, k]
end

ab = MSSTA.automul(a, b; tag_row="x", tag_shared="y", tag_col="z", alg="naive")
ab_arr_reconst = _tomat3(ab)
@assert ab_arr ≈ ab_arr_reconst